# Data Exploration
## Objectives
* Use Alpha API to collect relevant stock data
* Save all data into a single csv table
* Automate data collection

In [1]:
import requests 
import pandas as pd
import json
import numpy as np


In [90]:
class stock:
    def __init__(self, stock): 
        self.stock = stock
        self.yahoo_call = 0
        self.alpha_call = 0
        pass
    def get_prices(self):
        self.alpha_call +=1
        url = "https://alpha-vantage.p.rapidapi.com/query"

        querystring = {"interval":"5min","function":"TIME_SERIES_INTRADAY","symbol":self.stock,"datatype":"json","output_size":"compact"}

        headers = {
            'x-rapidapi-key': "4d83c30c33msh6c95f295970c8eep1bfc57jsn67c28d5a17a2",
            'x-rapidapi-host': "alpha-vantage.p.rapidapi.com"
            }

        response = requests.request("GET", url, headers=headers, params=querystring)
        data = json.loads(response.text)
        price_df = pd.DataFrame.from_dict(data['Time Series (5min)'], orient='index')
        price_df.index = pd.to_datetime(price_df.index)
        self.price_df = price_df
        
    def get_sma(self):
        self.alpha_call += 1
        url = "https://alpha-vantage.p.rapidapi.com/query"

        querystring = {"interval":"5min","series_type":"close","function":"SMA","symbol":self.stock,"time_period":"60","datatype":"json"}

        headers = {
        'x-rapidapi-key': "4d83c30c33msh6c95f295970c8eep1bfc57jsn67c28d5a17a2",
        'x-rapidapi-host': "alpha-vantage.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)
        data = json.loads(response.text)
        sma_df = pd.DataFrame.from_dict(data["Technical Analysis: SMA"], orient='index')
        sma_df.index = pd.to_datetime(sma_df.index)
        self.sma_df = sma_df
    
    def get_balance_sheet(self):
        self.yahoo_call += 1
        url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/qu/quote/" + self.stock + "/balance-sheet"

        headers = {
            'x-rapidapi-key': "4d83c30c33msh6c95f295970c8eep1bfc57jsn67c28d5a17a2",
            'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com"
            }

        response = requests.request("GET", url, headers=headers)
        data = json.loads(response.text)
        self.balance_sheet_df = pd.DataFrame.from_dict(data['balanceSheetHistory']['balanceSheetStatements'][0])
        self.balance_sheet_df = self.balance_sheet_df.drop(['fmt', 'longFmt'], axis=0)
    
    def get_cash_flow(self):
        self.yahoo_call += 1
        url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/qu/quote/" + self.stock + "/cashflow-statement"

        headers = {
            'x-rapidapi-key': "4d83c30c33msh6c95f295970c8eep1bfc57jsn67c28d5a17a2",
            'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com"
            }

        response = requests.request("GET", url, headers=headers)
        
        data = json.loads(response.text)
        
        df = None
        for i in range(len(data['cashflowStatementHistory']['cashflowStatements'])):
            new_df = pd.DataFrame.from_dict(data['cashflowStatementHistory']['cashflowStatements'][i])
            if (new_df.index == 'fmt').any():
                new_df = new_df.drop(['fmt', 'longFmt'], axis=0)
                new_df = new_df.drop(['endDate', 'maxAge'], axis=1)
            if df is not None:
                df = pd.concat([df, new_df])
            else: 
                df = new_df
            
        self.cash_flow_df =  pd.DataFrame(df.apply(np.mean, axis=0)).T
        
    def get_data(self):
        self.get_prices()
        self.get_sma()
        self.get_balance_sheet()
        self.get_cash_flow()


        self.merge1 = self.price_df.merge(self.sma_df, left_index=True, right_index=True)

        merge2 = self.balance_sheet_df.append(self.sma_df)
        merge2 = merge2.fillna(method='ffill')
        self.merge2 = merge2.drop('raw', axis=0)
        
        self.merge1['datetime'] = self.merge1.index
        sizeof = len(self.merge1)
        self.merge1.insert(1, "new_index", np.arange(sizeof))
        self.merge1 = self.merge1.set_index('new_index', drop=True)
        
        df = pd.DataFrame(data=self.merge2.tail(1), index=np.arange(100-1))
        df = df.append(self.merge2.tail(1))
        df = df.fillna(method='bfill')
        self.full_data = pd.merge(df, self.merge1)
        return self.full_data



* Take the size of the 5 minute intervals
* Take last of the merge2
* extend merge2 dataframe to size of merge1
* join.

== always 100